## Agent with GitHub Codespaces Integration

This notebook demonstrates an AI agent that can interact with GitHub Codespaces API to view and manage your codespaces.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
import json
from openai import AzureOpenAI
from helpers import function_to_schema
from github import Github
import requests

client = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],    
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview",
)

## GitHub Codespaces Functions

Define functions that can interact with GitHub Codespaces API

In [ ]:
def list_codespaces(token: str) -> str:
    """
    List all codespaces for the authenticated user.
    """
    try:
        headers = {
            'Authorization': f'token {token}',
            'Accept': 'application/vnd.github.v3+json'
        }
        
        response = requests.get('https://api.github.com/user/codespaces', headers=headers)
        
        if response.status_code == 200:
            codespaces_data = response.json()
            codespaces = codespaces_data.get('codespaces', [])
            
            if not codespaces:
                return "No codespaces found for your account."
            
            result = "Your Codespaces:\n\n"
            for idx, codespace in enumerate(codespaces, 1):
                result += f"{idx}. **{codespace['display_name'] or codespace['name']}**\n"
                result += f"   - Repository: {codespace['repository']['full_name']}\n"
                result += f"   - State: {codespace['state']}\n"
                result += f"   - Machine: {codespace['machine']['display_name']}\n"
                result += f"   - Created: {codespace['created_at'][:10]}\n"
                if codespace.get('web_url'):
                    result += f"   - URL: {codespace['web_url']}\n"
                result += "\n"
            
            return result
        else:
            return f"Error fetching codespaces: {response.status_code} - {response.text}"
    
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
def get_codespace_details(token: str, codespace_name: str) -> str:
    """
    Get detailed information about a specific codespace.
    """
    try:
        headers = {
            'Authorization': f'token {token}',
            'Accept': 'application/vnd.github.v3+json'
        }
        
        response = requests.get(f'https://api.github.com/user/codespaces/{codespace_name}', headers=headers)
        
        if response.status_code == 200:
            codespace = response.json()
            
            result = f"**Codespace Details: {codespace['display_name'] or codespace['name']}**\n\n"
            result += f"- Repository: {codespace['repository']['full_name']}\n"
            result += f"- State: {codespace['state']}\n"
            result += f"- Machine: {codespace['machine']['display_name']} ({codespace['machine']['name']})\n"
            result += f"- Storage: {codespace['machine']['storage_in_bytes'] // (1024**3)} GB\n"
            result += f"- CPUs: {codespace['machine']['cpus']}\n"
            result += f"- Memory: {codespace['machine']['memory_in_bytes'] // (1024**2)} MB\n"
            result += f"- Created: {codespace['created_at']}\n"
            result += f"- Last used: {codespace['last_used_at']}\n"
            
            if codespace.get('web_url'):
                result += f"- Web URL: {codespace['web_url']}\n"
            
            if codespace.get('git_status'):
                git_status = codespace['git_status']
                result += f"\n**Git Status:**\n"
                result += f"- Ref: {git_status.get('ref', 'N/A')}\n"
                result += f"- Ahead: {git_status.get('ahead', 0)} commits\n"
                result += f"- Behind: {git_status.get('behind', 0)} commits\n"
            
            return result
        else:
            return f"Error fetching codespace details: {response.status_code} - {response.text}"
    
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
def get_user_info(token: str) -> str:
    """
    Get information about the authenticated GitHub user.
    """
    try:
        headers = {
            'Authorization': f'token {token}',
            'Accept': 'application/vnd.github.v3+json'
        }
        
        response = requests.get('https://api.github.com/user', headers=headers)
        
        if response.status_code == 200:
            user = response.json()
            return f"Authenticated as: {user['login']} ({user.get('name', 'No name set')})"
        else:
            return f"Error fetching user info: {response.status_code}"
    
    except Exception as e:
        return f"Error: {str(e)}"

## Create GitHub Codespaces Agent

In [ ]:
# Create an assistant with GitHub Codespaces capabilities
codespaces_agent = client.beta.assistants.create(
    name="GitHub Codespaces Agent",
    instructions="""You are a helpful AI assistant that can help users manage and view their GitHub Codespaces.
    You have access to functions that can:
    1. List all codespaces for a user
    2. Get detailed information about a specific codespace
    3. Get user information
    
    When a user asks to view their codespaces, use the list_codespaces function.
    When they want details about a specific codespace, use get_codespace_details function.
    Always be helpful and provide clear, formatted information about their codespaces.
    
    Important: You will need a GitHub personal access token to access codespaces information.
    Make sure to remind users about this requirement.""",
    tools=[
        function_to_schema(list_codespaces),
        function_to_schema(get_codespace_details),
        function_to_schema(get_user_info)
    ],
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
)

## Test the Agent

**Note**: To use this agent, you'll need a GitHub Personal Access Token with `codespace` permissions. 
You can create one at: https://github.com/settings/tokens

Set your token in the environment variable `GITHUB_TOKEN` or pass it directly to the functions.

In [ ]:
import time
from IPython.display import clear_output

def handle_tool_calls(run, thread_id, github_token):
    """Handle tool calls from the assistant"""
    tool_outputs = []
    
    for tool_call in run.required_action.submit_tool_outputs.tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        print(f"Calling function: {function_name}")
        
        if function_name == "list_codespaces":
            result = list_codespaces(github_token)
        elif function_name == "get_codespace_details":
            result = get_codespace_details(github_token, arguments.get('codespace_name'))
        elif function_name == "get_user_info":
            result = get_user_info(github_token)
        else:
            result = f"Unknown function: {function_name}"
        
        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": result
        })
    
    return tool_outputs

In [ ]:
# Test the agent
# Replace 'your_github_token_here' with your actual GitHub token
github_token = os.getenv('GITHUB_TOKEN', 'your_github_token_here')

if github_token == 'your_github_token_here':
    print("⚠️  Please set your GitHub token in the GITHUB_TOKEN environment variable or replace the token above.")
    print("You can create a token at: https://github.com/settings/tokens")
    print("Make sure to include 'codespace' permissions.")
else:
    # Create a thread and test the agent
    thread = client.beta.threads.create()
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content="Please show me my codespaces"
    )
    
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=codespaces_agent.id,
    )
    
    # Wait for the run to complete and handle tool calls
    start_time = time.time()
    while run.status in ["queued", "in_progress", "requires_action"]:
        if run.status == "requires_action":
            tool_outputs = handle_tool_calls(run, thread.id, github_token)
            run = client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )
        
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        
        elapsed_time = time.time() - start_time
        clear_output(wait=True)
        print(f"Status: {run.status}")
        print(f"Elapsed time: {int(elapsed_time // 60)} minutes {int(elapsed_time % 60)} seconds")
    
    if run.status == "completed":
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        for message in reversed(messages.data):
            print(f"\n{message.role.upper()}: {message.content[0].text.value}")
    else:
        print(f"Run failed with status: {run.status}")
        if run.last_error:
            print(f"Error: {run.last_error.message}")